In [8]:
import pandas as pd

# Load the data from the provided Excel file
file_path = 'e64bc2d6-09fb-41a4-a0e2-d5f6412af7aa.xlsx'
df = pd.read_excel(file_path)

# Group by Shipment No to find the first and last leg for each shipment
shipment_groups = df.groupby('Shipment No').agg(
    first_leg_origin=pd.NamedAgg(column='Leg Origin Code', aggfunc=lambda x: x.iloc[0]),
    last_leg_destination=pd.NamedAgg(column='Leg Destination Code', aggfunc=lambda x: x.iloc[-1]),
    weight=pd.NamedAgg(column='Actual Weight (kg)', aggfunc=lambda x: x.iloc[0])  # Use the first leg's weight
).reset_index()

# Rename columns for clarity
shipment_groups = shipment_groups.rename(columns={
    'first_leg_origin': 'Origin',
    'last_leg_destination': 'Destination',
    'weight': 'Weight (kg)'
})

# Group by Origin-Destination pair and calculate the sum of shipments and weights
result = shipment_groups.groupby(['Origin', 'Destination']).agg(
    num_shipments=pd.NamedAgg(column='Shipment No', aggfunc='count'),
    total_weight=pd.NamedAgg(column='Weight (kg)', aggfunc='sum')
).reset_index()

# Calculate the total weight for all shipments
total_weight = result['total_weight'].sum()

# Add a column for the ratio of the specific tradelane's weight to the total weight
result['Weight Ratio'] = result['total_weight'] / total_weight

# Sort by the highest total weight per airport pair
result = result.sort_values(by='total_weight', ascending=False)

# Create an Excel writer object
output_file_path = 'processed_shipments.xlsx'
with pd.ExcelWriter(output_file_path, engine='xlsxwriter') as writer:
    # Write the aggregated result to the first sheet
    result.to_excel(writer, sheet_name='Aggregated Data', index=False)
    
    # Write the detailed shipment data to the second sheet
    shipment_groups.to_excel(writer, sheet_name='Shipment Details', index=False)

print(f'Processed data has been saved to {output_file_path}')


Processed data has been saved to processed_shipments.xlsx


: 